In [1]:
import os
#資料處理套件
import pandas as pd
import numpy as np
#聲音處理套件
import librosa
import librosa.display
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchinfo import summary
import torch.nn.functional as F
#random
import random
#BEATs模型
from BEATs import BEATs, BEATsConfig
#時間
import time

In [2]:
#存檔位置
ModelMode = "BEATs_iter3_ESC50"
ModelSetting = "notFbank_Paper_ProgressiveUnfreezing_kFold_normal"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[1] + "_" + ModelSetting + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[1] + "_" + ModelSetting + "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[1] + ModelSetting

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "/home/s1811032014/code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "/home/s1811032014/code/ESC-50-master/ESC-50-master/audio/"

In [ ]:
def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)
    #篩選出ESC10作為訓練資料
    df_10 = df[df['esc10']==True]
    #取出類別，去除重複並排序
    classes = df_10['category'].unique()
    class_dict = {i:x for x,i in enumerate(classes)}
    #重新標label
    df_10.loc[:, 'target'] = df_10['category'].map(class_dict)
    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df_10[df_10.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                dataX.append(sound)
                dataY.append(data[3])
                
            count +=1
            print("{:.3f}".format(100 * count / len(df_10))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]
    
    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#BEATs_Pre-Train_iter3模型
class BEATs_Pre_Train_iter3(nn.Module):
    def __init__(self, nums_class=10):
        super(BEATs_Pre_Train_iter3,self).__init__()
        
        #載入模型BEATs
        checkpoint = torch.load('BEATs_iter3.pt')
        cfg = BEATsConfig(checkpoint['cfg'])
        BEATs_model = BEATs(cfg)
        BEATs_model.load_state_dict(checkpoint['model'])
        
        #BEATs
        self.BEATs = BEATs_model
        
        #丟棄層
        self.last_Dropout = nn.Dropout(0.1)
        
        #全連接層
        self.last_layer = nn.Linear(cfg.encoder_embed_dim, nums_class)   
        
    def forward(self,x):
        #BEATs
        output = self.BEATs.extract_features(x)[0]
        #丟棄層
        output = self.last_Dropout(output)
        #全連接層
        output = self.last_layer(output)
        #mean
        output = output.mean(dim=1)
        #sigmoid
        output = torch.sigmoid(output) 
        return output

In [ ]:
#訓練模型(只訓練分類器)
def train_model_TrainClassifier(trainFold):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 300
    #設置總訓練 warm-up 的 epochs
    warmup_epochs = 100
    #批次大小
    BATCH_SIZE = 30
    #學習率
    LR = 3e-4
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #過濾凍結的參數層
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    #優化器
    optimizer = torch.optim.AdamW(parameters, lr=LR, betas=(0.9, 0.98), weight_decay=0.01)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler 
    #前1/10次 epochs用 warn-up epochs方法
    #後面用 Cosine Annealing方法
    #--------------------------------------------------------------------
    # 定義 lambda 函數來調整學習率
    lr_lambda = lambda epoch: min((epoch+1) / warmup_epochs, 1)
    # 使用 LambdaLR 來設置學習率調整器
    scheduler_warmup = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
    # 使用 余弦退火（Cosine Annealing） 來設置學習率調整器
    scheduler_consine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=400)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #warm-up
            if epoch < warmup_epochs:
                scheduler_warmup.step(epoch)
                # 獲取當前的學習率
                current_lr = optimizer.param_groups[0]['lr']
                sch_name = "warmup"
                
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        if (epoch >= (warmup_epochs)):
            scheduler_consine.step()
            # 獲取當前的學習率
            current_lr = optimizer.param_groups[0]['lr']
            sch_name = "consine"
            
        # 打印當前的學習率
        formatted_lr = f"{current_lr:.10f}"
        print(f"Epoch [{epoch+1}/{num_epochs}] - scheduler_fun: {sch_name} _ Learning Rate: {formatted_lr}")
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [ ]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 30
    #設置總訓練 warm-up 的 epochs
    warmup_epochs = 10
    #批次大小
    BATCH_SIZE = 15
    #學習率
    LR = 3e-5
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #過濾凍結的參數層
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    #優化器
    optimizer = torch.optim.AdamW(parameters, lr=LR, betas=(0.9, 0.98), weight_decay=0.01)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler 
    #前1/10次 epochs用 warn-up epochs方法
    #後面用 Cosine Annealing方法
    #--------------------------------------------------------------------
    # 定義 lambda 函數來調整學習率
    lr_lambda = lambda epoch: min((epoch+1) / warmup_epochs, 1)
    # 使用 LambdaLR 來設置學習率調整器
    scheduler_warmup = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
    # 使用 余弦退火（Cosine Annealing） 來設置學習率調整器
    scheduler_consine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #warm-up
            if epoch < warmup_epochs:
                scheduler_warmup.step(epoch)
                # 獲取當前的學習率
                current_lr = optimizer.param_groups[0]['lr']
                sch_name = "warmup"
                
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        if (epoch >= (warmup_epochs)):
            scheduler_consine.step()
            # 獲取當前的學習率
            current_lr = optimizer.param_groups[0]['lr']
            sch_name = "consine"
            
        # 打印當前的學習率
        formatted_lr = f"{current_lr:.10f}"
        print(f"Epoch [{epoch+1}/{num_epochs}] - scheduler_fun: {sch_name} _ Learning Rate: {formatted_lr}")
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [ ]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [ ]:
kFoldTrainTimePart1 = []
kFoldTrainTimePart2 = []
kFoldTrainAccuracyPart1 = []
kFoldTrainAccuracyPart2 = []
kFoldTestAccuracy1Part1 = []
kFoldTestAccuracy1Part2 = []
kFoldTestAccuracy3Part1 = []
kFoldTestAccuracy3Part2 = []
kFoldTestAccuracy5Part1 = []
kFoldTestAccuracy5Part2 = []

#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = BEATs_Pre_Train_iter3(nums_class = 10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #訓練資料(要用幾秒訓練)
    dataX = dataX3
    dataY = dataY3

    #--------------------------------------------------------------------
    # 鎖住權重只訓練最後一層
    #--------------------------------------------------------------------
    #鎖住權重
    #最後一層可訓練
    for name, param in model.named_parameters():
        if name not in ['module.last_layer.weight', 'module.last_layer.bias']:
            param.requires_grad = False
        else:
            param.requires_grad = True

    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型(只訓練分類器)
    part1_train_loss, part1_train_accuracy, part1_test_accuracy1, part1_test_accuracy3, part1_test_accuracy5 = train_model_TrainClassifier(fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    part1_training_time = end_time - start_time

    #印出訓練耗時
    print("Part1 Training Time: {:.2f} seconds".format(part1_training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("第一階段訓練\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(part1_train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss_Part1: {} Train_Acc_Part1: {:.6f}%\n".format(fold, i+1, len(part1_train_loss), part1_train_loss[i], part1_train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1_Part1: {:.6f}% Test_Accuracy3_Part1: {:.6f}% Test_Accuracy5_Part1: {:.6f}%\n".format(part1_test_accuracy1, part1_test_accuracy3, part1_test_accuracy5))
    f.write("Part1 Training Time: {:.2f} seconds\n".format(part1_training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    #--------------------------------------------------------------------
    #訓練模型(全部)
    #解開鎖住的權重
    #--------------------------------------------------------------------
    for name, param in model.named_parameters():
        param.requires_grad = True

    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型(全部)
    part2_train_loss, part2_train_accuracy, part2_test_accuracy1, part2_test_accuracy3, part2_test_accuracy5 = train_model_TrainAll(fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    part2_training_time = end_time - start_time

    #印出訓練耗時
    print("Part2 Training Time: {:.2f} seconds".format(part2_training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("第二階段訓練\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(part2_train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss_Part2: {} Train_Acc_Part2: {:.6f}%\n".format(fold, i+1, len(part2_train_loss), part2_train_loss[i], part2_train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1_Part2: {:.6f}% Test_Accuracy3_Part2: {:.6f}% Test_Accuracy5_Part2: {:.6f}%\n".format(part2_test_accuracy1, part2_test_accuracy3, part2_test_accuracy5))
    f.write("Part2 Training Time: {:.2f} seconds\n".format(part2_training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    #-----------------------------------------------------------------------
    
    kFoldTrainTimePart1.append(part1_training_time)
    kFoldTrainTimePart2.append(part2_training_time)
    kFoldTrainAccuracyPart1.append(part1_train_accuracy[-1])
    kFoldTrainAccuracyPart2.append(part2_train_accuracy[-1])
    kFoldTestAccuracy1Part1.append(part1_test_accuracy1)
    kFoldTestAccuracy1Part2.append(part2_test_accuracy1)
    kFoldTestAccuracy3Part1.append(part1_test_accuracy3)
    kFoldTestAccuracy3Part2.append(part2_test_accuracy3)
    kFoldTestAccuracy5Part1.append(part1_test_accuracy5)
    kFoldTestAccuracy5Part2.append(part2_test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

In [ ]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("第一階段訓練\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracyPart1[fold], kFoldTestAccuracy1Part1[fold], kFoldTestAccuracy3Part1[fold], kFoldTestAccuracy5Part1[fold], kFoldTrainTimePart1[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracyPart1), max(kFoldTestAccuracy1Part1), max(kFoldTestAccuracy3Part1), max(kFoldTestAccuracy5Part1)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracyPart1), min(kFoldTestAccuracy1Part1), min(kFoldTestAccuracy3Part1), min(kFoldTestAccuracy5Part1)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracyPart1), np.mean(kFoldTestAccuracy1Part1), np.mean(kFoldTestAccuracy3Part1), np.mean(kFoldTestAccuracy5Part1)))

f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("第二階段訓練\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracyPart2[fold], kFoldTestAccuracy1Part2[fold], kFoldTestAccuracy3Part2[fold], kFoldTestAccuracy5Part2[fold], kFoldTrainTimePart2[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracyPart2), max(kFoldTestAccuracy1Part2), max(kFoldTestAccuracy3Part2), max(kFoldTestAccuracy5Part2)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracyPart2), min(kFoldTestAccuracy1Part2), min(kFoldTestAccuracy3Part2), min(kFoldTestAccuracy5Part2)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracyPart2), np.mean(kFoldTestAccuracy1Part2), np.mean(kFoldTestAccuracy3Part2), np.mean(kFoldTestAccuracy5Part2)))

f.close()